In [2]:
# 실제 train data를 변환하기 - 데이터 읽어들이기

import json
import os
import pandas as pd
import numpy as np

base_dir = os.getcwd()

train_images_path = base_dir + '/data/train/images/'
train_labels_json_path = base_dir + '/data/train/labels_json/'
train_labels_path = base_dir + '/data/train/labels/'

os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_json_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)

!unzip -qq '/content/drive/MyDrive/KCC3기_2차_프로젝트/train.zip' -d '/content/data/train'

train_images_list = os.listdir(train_images_path)
train_labels_json_list = os.listdir(train_labels_json_path)

train_images_list.sort()
train_labels_json_list.sort()

In [3]:
# 실제 train data를 변환하기 - 라벨링 변환

yolo_class = {
    '철캔': 0,
    '알루미늄캔': 4,
    '종이': 8,
    '무색단일': 12,
    '유색단일': 16,
    'PE': 20,
    'PP': 24,
    'PS': 28,
    '스티로폼': 32,
    '비닐': 36,
    '갈색': 40,
    '녹색': 44,
    '투명': 48
}

for label in train_labels_json_list:
  file_json = open(train_labels_json_path + label)
  f_json = json.load(file_json)

  img_width = f_json['IMAGE_INFO']['IMAGE_WIDTH']
  img_height = f_json['IMAGE_INFO']['IMAGE_HEIGHT']

  f_txt = open(train_labels_path + label.replace('.json', '.txt'), 'w')

  for anno in f_json['ANNOTATION_INFO']:
    obj_id = yolo_class[anno['DETAILS']]

    if anno['DAMAGE'] != '원형':
      obj_id += 2
    if anno['DIRTINESS'] != '오염없음':
      obj_id += 1

    obj_points = np.array(anno['POINTS'])
    if obj_points[0].size == 4:
      yolo_id = obj_id
      yolo_points = np.array([obj_points[0][0]+obj_points[0][2]/2, obj_points[0][1]+obj_points[0][3]/2,
                              obj_points[0][2], obj_points[0][3]])

      yolo_points_norm = np.array([yolo_points[0]/img_width, yolo_points[1]/img_height,
                             yolo_points[2]/img_width, yolo_points[3]/img_height])

      yolo_points_norm = list(yolo_points_norm)
      f_txt.write(str(yolo_id) + ' ')
      f_txt.write(' '.join(str(ele) for ele in yolo_points_norm))
      f_txt.write('\n')
    elif obj_points.size >= 2:
      max_x = 0
      min_x = img_width
      max_y = 0
      min_y = img_height
      for points in obj_points:
        if points[0] > max_x:
          max_x = points[0]
        if points[0] < min_x:
          min_x = points[0]
        if points[1] > max_y:
          max_y = points[1]
        if points[1] < min_y:
          min_y = points[1]

      yolo_id = obj_id
      yolo_points = np.array([(min_x+max_x)/2, (min_y+max_y)/2,
                              max_x-min_x, max_y-min_y])
      yolo_points_norm = np.array([yolo_points[0]/img_width, yolo_points[1]/img_height,
                             yolo_points[2]/img_width, yolo_points[3]/img_height])
      yolo_points_norm = list(yolo_points_norm)

      f_txt.write(str(yolo_id) + ' ')
      f_txt.write(' '.join(str(ele) for ele in yolo_points_norm))
      f_txt.write('\n')
    else:
      print(label)
  f_txt.close()


In [4]:
# json 라벨 삭제 및 라벨링 확인

import shutil

shutil.rmtree('/content/data/train/labels_json')

print(open('/content/data/train/labels/606511@4_04002_220907_P1_T1.txt').readline())
print(open('/content/data/train/labels/10349@3_01001_220715_P1_T1.txt').readline())

24 0.4915269230769231 0.49103166345617255 0.38315604395604386 0.6721641025641029

0 0.6400563522733375 0.3877540579713878 0.1944409271364939 0.387188312642922



In [5]:
# validation 데이터 분리

import shutil

val_images_path = base_dir + '/data/val/images/'
val_labels_path = base_dir + '/data/val/labels/'

os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

train_images_list = os.listdir(train_images_path)
train_labels_list = os.listdir(train_labels_path)

train_images_list.sort()
train_labels_list.sort()

for i in range(2600):
  shutil.move(train_images_path + train_images_list[i*10], val_images_path)
  shutil.move(train_labels_path + train_labels_list[i*10], val_labels_path)


In [6]:
# 분리한 데이터 이미지와 라벨 파일 일치 여부 확인

print(len(os.listdir('/content/data/train/images/')))
print(len(os.listdir('/content/data/train/labels/')))

print(len(os.listdir('/content/data/val/images/')))
print(len(os.listdir('/content/data/val/labels/')))

train_images_list = os.listdir(train_images_path)
train_labels_list = os.listdir(train_labels_path)

train_images_list.sort()
train_labels_list.sort()

for i in range(len(train_images_list)):
  if train_images_list[i].replace('.jpg', '') != train_labels_list[i].replace('.txt', ''):
    print(train_images_list[i])

val_images_list = os.listdir(val_images_path)
val_labels_list = os.listdir(val_labels_path)

val_images_list.sort()
val_labels_list.sort()

for i in range(len(val_images_list)):
  if val_images_list[i].replace('.jpg', '') != val_labels_list[i].replace('.txt', ''):
    print(val_images_list[i])


23400
23400
2600
2600


In [16]:
# YOLO 학습을 위한 yaml 파일 생성

!pip install PyYAML

import yaml

data = {
    'train': '/content/data/train/images/',
    'val': '/content/data/val/images',
    'names': ['IronCan_NoDamNoDir', 'IronCan_NoDamDir', 'IronCan_DamNoDir', 'IronCan_DamDir',
              'AluminumCan_NoDamNoDir', 'AluminumCan_NoDamDir', 'AluminumCan_DamNoDir', 'AluminumCan_DamDir',
              'Paper_NoDamNoDir', 'Paper_NoDamDir', 'Paper_DamNoDir', 'Paper_DamDir',
              'PetNoColor_NoDamNoDir', 'PetNoColor_NoDamDir', 'PetNoColor_DamNoDir', 'PetNoColor_DamDir',
              'PetColor_NoDamNoDir', 'PetColor_NoDamDir', 'PetColor_DamNoDir', 'PetColor_DamDir',
              'PE_NoDamNoDir', 'PE_NoDamDir', 'PE_DamNoDir', 'PE_DamDir',
              'PP_NoDamNoDir', 'PP_NoDamDir', 'PP_DamNoDir', 'PP_DamDir',
              'PS_NoDamNoDir', 'PS_NoDamDir', 'PS_DamNoDir', 'PS_DamDir',
              'Styrofoam_NoDamNoDir', 'Styrofoam_NoDamDir', 'Styrofoam_DamNoDir', 'Styrofoam_DamDir',
              'Vinyl_NoDamNoDir', 'Vinyl_NoDamDir', 'Vinyl_DamNoDir', 'Vinyl_DamDir',
              'GlassBrown_NoDamNoDir', 'GlassBrown_NoDamDir', 'GlassBrown_DamNoDir', 'GlassBrown_DamDir',
              'GlassGreen_NoDamNoDir', 'GlassGreen_NoDamDir', 'GlassGreen_DamNoDir', 'GlassGreen_DamDir',
              'GlassTransparency_NoDamNoDir', 'GlassTransparency_NoDamDir', 'GlassTransparency_DamNoDir', 'GlassTransparency_DamDir'],
    'nc': 52
}

yaml_name = 'recycle_data.yaml'
with open('/content/data/' + 'recycle_data.yaml', 'w') as f:
  yaml.dump(data, f)

with open('/content/data/' + 'recycle_data.yaml', 'r') as f:
  recycle_yaml = yaml.safe_load(f)
  display(recycle_yaml)

{'names': ['IronCan_NoDamNoDir',
  'IronCan_NoDamDir',
  'IronCan_DamNoDir',
  'IronCan_DamDir',
  'AluminumCan_NoDamNoDir',
  'AluminumCan_NoDamDir',
  'AluminumCan_DamNoDir',
  'AluminumCan_DamDir',
  'Paper_NoDamNoDir',
  'Paper_NoDamDir',
  'Paper_DamNoDir',
  'Paper_DamDir',
  'PetNoColor_NoDamNoDir',
  'PetNoColor_NoDamDir',
  'PetNoColor_DamNoDir',
  'PetNoColor_DamDir',
  'PetColor_NoDamNoDir',
  'PetColor_NoDamDir',
  'PetColor_DamNoDir',
  'PetColor_DamDir',
  'PE_NoDamNoDir',
  'PE_NoDamDir',
  'PE_DamNoDir',
  'PE_DamDir',
  'PP_NoDamNoDir',
  'PP_NoDamDir',
  'PP_DamNoDir',
  'PP_DamDir',
  'PS_NoDamNoDir',
  'PS_NoDamDir',
  'PS_DamNoDir',
  'PS_DamDir',
  'Styrofoam_NoDamNoDir',
  'Styrofoam_NoDamDir',
  'Styrofoam_DamNoDir',
  'Styrofoam_DamDir',
  'Vinyl_NoDamNoDir',
  'Vinyl_NoDamDir',
  'Vinyl_DamNoDir',
  'Vinyl_DamDir',
  'GlassBrown_NoDamNoDir',
  'GlassBrown_NoDamDir',
  'GlassBrown_DamNoDir',
  'GlassBrown_DamDir',
  'GlassGreen_NoDamNoDir',
  'GlassGreen_NoDamD

In [8]:
# YOLO 학습 위한 환경

%pip install ultralytics

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.156 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 36.5/78.2 GB disk)


In [9]:
# YOLO 전이 학습
from ultralytics import YOLO
import shutil, os

model = YOLO('yolov8n.pt')

model.train(data='/content/data/recycle_data.yaml', epochs=1, patience=3, batch=64, imgsz=640, name='recycle', exist_ok=True)

if os.path.isdir('/content/drive/MyDrive/KCC_2nd_project/models/recycle'):
  shutil.rmtree('/content/drive/MyDrive/KCC_2nd_project/models/recycle')

shutil.copytree('/content/runs/detect/recycle/' , '/content/drive/MyDrive/KCC_2nd_project/models/recycle/')

100%|██████████| 6.23M/6.23M [00:00<00:00, 57.0MB/s]
Ultralytics YOLOv8.0.156 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data/recycle_data.yaml, epochs=1, patience=3, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=recycle, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, f

'/content/drive/MyDrive/KCC_2nd_project/models/recycle'

In [13]:
# google drive의 모델 파일을 불러들여 다시 전이 학습
# 학습 완료 시 다시 google drive로 모델 파일 이동

from ultralytics import YOLO
import shutil, os

model = YOLO('/content/drive/MyDrive/KCC_2nd_project/models/recycle/weights/best.pt')

model.train(data='/content/data/recycle_data.yaml', epochs=3, patience=3, batch=64, imgsz=640, name='recycle', exist_ok=True)

if os.path.isdir('/content/drive/MyDrive/KCC_2nd_project/models/recycle'):
  shutil.rmtree('/content/drive/MyDrive/KCC_2nd_project/models/recycle')

shutil.copytree('/content/runs/detect/recycle/' , '/content/drive/MyDrive/KCC_2nd_project/models/recycle/')

Ultralytics YOLOv8.0.156 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/KCC_2nd_project/models/recycle/weights/best.pt, data=/content/data/recycle_data.yaml, epochs=1, patience=3, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=recycle, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=T

AssertionError: ignored

In [12]:
# 예측 값 확인

from ultralytics import YOLO
import shutil, os
from PIL import Image
import matplotlib.pyplot as plt

model = YOLO('/content/drive/MyDrive/KCC_2nd_project/models/recycle/weights/best.pt')


for i in range(10):
  results = model('/content/data/val/images/' + os.listdir('/content/data/val/images/')[i])

  # Show the results
  for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    plt.imshow(im)
    plt.show()

Output hidden; open in https://colab.research.google.com to view.